In [0]:
# service principal for integrating with ADLS and access it's data

spark.conf.set("fs.azure.account.auth.type.hpadlsacc.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.hpadlsacc.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.hpadlsacc.dfs.core.windows.net", dbutils.secrets.get("hc-secret-scope", "app-key"))
spark.conf.set("fs.azure.account.oauth2.client.secret.hpadlsacc.dfs.core.windows.net", dbutils.secrets.get("hc-secret-scope", "service-cred"))
tenant_id = dbutils.secrets.get("hc-secret-scope", "dir-id")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.hpadlsacc.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

# creating dimesional provider table

In [0]:
%sql
-- creating a dim privider table from the silver.providers table

CREATE TABLE IF NOT EXISTS gold.dim_provider (
  ProviderID string,
  FirstName string,
  LastName string,
  DeptID string,
  datasource string
)
USING DELTA
LOCATION "abfss://gold@hpadlsacc.dfs.core.windows.net/dim_provider";


In [0]:
%sql
-- truncate the table
truncate TABLE gold.dim_provider;


In [0]:
%sql
-- inserting data
INSERT INTO gold.dim_provider
SELECT 
  ProviderID,
  FirstName,
  LastName,
  CONCAT(DeptID, '-', datasource) AS DeptID,
  datasource
FROM silver.providers
WHERE is_quarantined = false;


In [0]:
%sql
-- display the table
select  * from gold.dim_provider;